In [1]:
import sys
sys.path.append("../")

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rc('text', usetex=False)

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import scipy

from rsep_explain.variables import auto_var
from utils import params_to_dataframe
from params import LinSepBBMRobExperiments3
from IPython.display import display

fontsize = 20

In [2]:
def get_df(exp, result_file_dir='./results/calc_separation', columns=None):
    _, _, grid_params, _ = exp()()
    df = params_to_dataframe(grid_params,
                             columns=columns,
                             result_file_dir=result_file_dir,
                             logging_level=0)
    return df

In [3]:
ori_df = get_df(LinSepBBMRobExperiments3, '../results/lin_sep_bbm_rob_3/', ['bbm_results'])

In [4]:
unique_clfs = {}
accs = {}
dset_curve = {}
dset_max_depth = {}
df = ori_df.loc[ori_df['rsep'] == 0.05].copy()
for i, d in df.iterrows():
    d['bbm_results']['dif_depth'].append({'tst acc': d['bbm_results']['tst acc']})
    #temp = np.asarray([data['tst acc'] for data in d["bbm_results"]['dif_depth'] if data])

    temp = []
    unique_leaners = set()
    for i in range(len(d['bbm_results']['learned_model'])):
        if d['bbm_results']['learned_model'][i] in unique_leaners:
            temp[-1] = max(temp[-1], d["bbm_results"]['dif_depth'][i]['tst acc'])
        else:
            temp.append(d["bbm_results"]['dif_depth'][i+1]['tst acc'])
        unique_leaners.add(d['bbm_results']['learned_model'][i])
        
    curve = np.zeros(100)
    curve[:len(temp)] = temp
    curve[len(temp):] = temp[-1]
    dset_max_depth[d['dataset']] = max(len(temp), dset_max_depth.setdefault(d['dataset'], 0))
    dset_curve.setdefault(d['dataset'], []).append(curve)

In [6]:
for k, v in dset_curve.items():
    #plt.title(k, fontsize=fontsize)
    #plt.plot(np.mean(v, axis=0), label="lin sep")
    #plt.plot(np.mean(dset_ori_curve[k], axis=0), label="ori")
    depth = dset_max_depth[k]
    plt.errorbar(x=np.arange(100)[:depth],
                 y=np.mean(np.array(dset_curve[k])[:, :depth], axis=0),
                 yerr=scipy.stats.sem(np.array(dset_curve[k])[:, :depth], axis=0))
    #plt.xlabel('# weak learners', fontsize=fontsize)
    plt.xlabel('interpretation complexity', fontsize=fontsize)
    plt.ylabel('test accuracy', fontsize=fontsize)
    #plt.legend(fontsize=fontsize)
    plt.tight_layout()
    plt.savefig(f"./figs/bbm_acc_number/{k}.png")
    plt.close()